**Исследование комфортных условий в офисах в разных странах**

Цель: Определить, в какой тип климатического оборудования стоит инвестировать больше, чтобы повысить удовлетворённость клиентов. Понять климатические особенности разных регионов, оценить комфорт в офисах и найти закономерности.

In [ ]:
# Импортируем основные библиотеки для анализа и визуализации
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import lstsq
import seaborn as sns
import os

# Настраиваем отображение всех колонок при выводе
pd.set_option("display.max_columns", None)

# Загружаем датасет (предварительно выгруженный в формате CSV)
df = pd.read_csv("Data.csv", delimiter=";")


Мы начинаем с загрузки данных, в которых содержатся измерения условий в офисах в разных странах, включая температуру, влажность, скорость воздуха, а также субъективные оценки комфорта.


**Подготовка данных**

In [ ]:
# Добавляем возрастные группы по заданным интервалам
df["Возрастная группа"] = pd.cut(
    df["Возраст"],
    bins=[0, 18, 30, 45, 60, 120],
    labels=["<18", "18-30", "30-45", "45-60", "60+"],
    right=False
)

# Разбиваем относительную влажность на категории
df["RH комфорт"] = pd.cut(
    df["RH"],
    bins=[0, 30, 60, 100],
    labels=["Сухо", "Комфорт", "Влажно"],
    right=False
)

# Убираем пробелы в названиях способов охлаждения
df["Способ охлаждения"] = df["Способ охлаждения"].astype(str).str.strip()

# Создаём папку для сохранения графиков, если её ещё нет
os.makedirs("plots", exist_ok=True)


Эти категории помогут нам анализировать предпочтения и комфорт среди разных возрастов и условий.

**Группировка по полу и возрасту. Средние значения**

In [ ]:
# Группировка по стране
for country in df["Страна"].dropna().unique():
    df_country = df[df["Страна"] == country].copy()

    summary = (
        df_country.dropna(subset=["Пол", "Возрастная группа", "Температура воздуха в помещении"])
        .groupby(["Пол", "Возрастная группа"], observed=True)
        .agg({
            "Температура воздуха в помещении": "mean",
            "Температура воздуха на улице": "mean",
            "RH": "mean"
        })
        .reset_index()
        .rename(columns={
            "Температура воздуха в помещении": "Средняя температура в помещении",
            "Температура воздуха на улице": "Средняя температура на улице",
            "RH": "Средняя влажность"
        })
    )

    display(summary)


Эта таблица позволяет увидеть климатический комфорт внутри и снаружи помещения, а также различия между мужчинами и женщинами разных возрастов.

**Комфортная влажность: соответствие стандарту**

Согласно рекомендациям ASHRAE и ГОСТ, оптимальный уровень относительной влажности в помещениях: 30–60%.

In [ ]:
# Оцениваем, соответствует ли RH стандарту комфорта
df["RH_стандарт"] = df["RH"].apply(lambda x: "В норме" if 30 <= x <= 60 else "Отклонение")


**Корреляционный анализ**

In [ ]:
corr_cols = [
    "Температура воздуха в помещении",
    "Температура воздуха на улице",
    "RH",
    "Скорость воздуха",
    "Оценка комфорта",
    "Среднемесячная температура на улице"
]

for country in df["Страна"].dropna().unique():
    df_country = df[df["Страна"] == country].copy()
    corr_df = df_country[corr_cols].dropna()

    if not corr_df.empty:
        corr = corr_df.corr()
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
        plt.title(f"Корреляция ({country})")
        plt.tight_layout()
        plt.savefig(f"plots/correlation_{country}.png")
        plt.close()


В большинстве стран наблюдается сильная положительная корреляция между температурой на улице и внутри офиса. Это означает, что системы охлаждения не всегда компенсируют внешний климат.

**Регрессионная модель: что влияет на внутреннюю температуру?**

In [ ]:
for country in df["Страна"].dropna().unique():
    df_country = df[df["Страна"] == country].copy()

    model_cols = [
        "Температура воздуха в помещении",
        "Температура воздуха на улице",
        "RH",
        "Скорость воздуха",
        "Оценка комфорта",
        "Способ охлаждения"
    ]
    df_model = df_country[model_cols].dropna()

    if not df_model.empty:
        X = pd.concat([
            df_model[["Температура воздуха на улице", "RH", "Скорость воздуха", "Оценка комфорта"]],
            pd.get_dummies(df_model["Способ охлаждения"], prefix="Охлаждение", dtype=float)
        ], axis=1)

        X = X.astype(float)
        y = df_model["Температура воздуха в помещении"].astype(float)
        X.insert(0, "Intercept", 1.0)

        beta = lstsq(X.values, y.values, rcond=None)[0]
        y_pred = X @ beta
        r2 = 1 - np.sum((y - y_pred) ** 2) / np.sum((y - np.mean(y)) ** 2)
        rmse = np.sqrt(np.mean((y - y_pred) ** 2))

        print(f"=== {country} ===")
        print("Коэффициенты регрессии:")
        for name, coef in zip(X.columns, beta):
            print(f"{name}: {coef:.3f}")
        print(f"R2: {r2:.4f}")
        print(f"RMSE: {rmse:.4f}")


Интерпретация:

Большие значения R² (близкие к 1) говорят о хорошем качестве модели.
Значимые коэффициенты у переменных типа охлаждения могут показать, какие методы наиболее эффективны в конкретных климатических условиях.

**Выводы**

1. Температура в офисе часто зависит от температуры на улице, особенно в странах с жарким климатом.
2. Влажность во многих случаях отклоняется от рекомендованной — важно инвестировать в увлажнители/осушители воздуха.
3. Тип охлаждения (например, центральный кондиционер, вентиляция и т.д.) имеет разный эффект в зависимости от региона.
4. Рекомендация: Инвестировать больше в региональные решения, например, мощное кондиционирование в жарких странах и системы вентиляции с контролем влажности в странах с колебаниями влажности.